# Tarefa: Iris - Tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 KB 1.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch.nn as nn
from skorch import NeuralNetClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

'1.13.1+cu116'

## Etapa 2: Base de dados

In [3]:
torch.manual_seed(123)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
base = pd.read_csv('/content/drive/MyDrive/Deep Learing de A à Z com PyTorch/Bases/iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)

In [7]:
previsores = previsores.astype('float32')
classe = classe.astype('int64')

## Etapa 3: Construção do modelo

In [8]:
class classificador_torch(nn.Module):
    def __init__(self, activation, neurons, initializer, dropout):
        super().__init__()
        self.dense0 = nn.Linear(4, neurons)
        initializer(self.dense0.weight)
        self.activation0 = activation
        self.dense1 = nn.Linear(neurons, neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation
        self.dense2 = nn.Linear(neurons, 3)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dropout(X)
        X = self.dense2(X)
        return X

In [9]:
classificador_sklearn = NeuralNetClassifier(module = classificador_torch,
                                            criterion = torch.nn.CrossEntropyLoss,
                                            train_split = False)

## Etapa 4: Tuning

In [10]:
params = {'batch_size': [10, 30],
          'max_epochs': [2000, 3000],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'module__activation': [F.relu, F.tanh, F.sigmoid],
          'module__neurons': [4, 8, 16],
          'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_],
          'module__dropout': [0.2, 0.3]
          }

In [11]:
grid_search = GridSearchCV(estimator = classificador_sklearn, 
                           param_grid = params, 
                           scoring = 'accuracy',
                           cv = 3)

In [ ]:
grid_search = grid_search.fit(previsores, classe)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_